In [1]:
import numpy as np
import pandas as pd
import torch
from torch import nn
import torch.nn.functional as F
!wget https://raw.githubusercontent.com/Bjarten/early-stopping-pytorch/master/pytorchtools.py
from pytorchtools import EarlyStopping

--2021-12-12 07:38:09--  https://raw.githubusercontent.com/Bjarten/early-stopping-pytorch/master/pytorchtools.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2109 (2.1K) [text/plain]
Saving to: ‘pytorchtools.py’

pytorchtools.py     100%[===================>]   2.06K  --.-KB/s    in 0s      

2021-12-12 07:38:09 (50.9 MB/s) - ‘pytorchtools.py’ saved [2109/2109]



In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [4]:
with open('/content/drive/MyDrive/dls/kaggle_poem_dataset.csv', 'r') as file:
  text = file.read()

In [5]:
class RNN_LSTM(nn.Module):        
  def __init__(self, dim):
    super().__init__()
    self.lstm_layer=nn.LSTM(dim,256,2, dropout=0.5,batch_first=True)
    self.dropout=nn.Dropout(0.5)
    self.linear_layer=nn.Linear(256,dim)

  def forward(self, x, hidden):
    r_output, hidden = self.lstm_layer(x, hidden)
    out = self.dropout(r_output)
    out = out.contiguous().view(-1, 256)
    out = self.linear_layer(out)
    return out, hidden

  def initialise(self, batch_size):
    initials = next(self.parameters()).data
    hidden = (initials.new(2, batch_size, 256).zero_().to(device),
              initials.new(2, batch_size, 256).zero_().to(device))
    return hidden

In [6]:
def load(encoded_Values, batchsize, s):
  batchsize_total = batchsize * s
  batches = len(encoded_Values)//batchsize_total
  encoded_Values = encoded_Values[:batches * batchsize_total]
  encoded_Values = encoded_Values.reshape((batchsize, -1))
  for n in range(0, encoded_Values.shape[1], s):
    inputs = encoded_Values[:, n:n+s]
    targets = np.zeros_like(inputs)
    if n+s<len(encoded_Values):
      targets[:, :-1], targets[:, -1] = inputs[:, 1:], encoded_Values[:, n+s]
    else:
      targets[:, :-1], targets[:, -1] = inputs[:, 1:], encoded_Values[:, 0]
    yield inputs, targets

In [7]:
def encode_onehot(data, n):
  encodedVal = np.zeros((data.size, n), dtype=np.float32)
  encodedVal[np.arange(encodedVal.shape[0]), data.flatten()] = 1.
  encodedVal = encodedVal.reshape((*data.shape, n))
  return encodedVal

In [8]:
chars = tuple(set(text))
charVersion = dict(enumerate(chars))
intVersion = {c: i for i, c in charVersion.items()}
encoded = np.array([intVersion[c] for c in text])

In [9]:
def train(model, encoded_Values, epochs=100, batchsize=10, s=50):
  early_stopping = EarlyStopping(patience=4, verbose=True)
  optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
  criterion = nn.CrossEntropyLoss()
  split = int(len(encoded_Values)*(0.9))
  trainData, testData = encoded_Values[:split], encoded_Values[split:]
  dim = len(chars);print(dim)
  for epoch in range(epochs):
      h = model.initialise(batchsize)
      trainLoss =[]
      for x, y in load(trainData, batchsize, s):
          x = encode_onehot(x, dim)
          inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
          inputs, targets = inputs.to(device), targets.to(device)
          h = tuple([each.data for each in h])
          optimizer.zero_grad()
          predicted, h = model(inputs, h)
          loss = criterion(predicted, targets.view(batchsize*s).long())
          loss.backward()
          trainLoss.append(loss.item())
          nn.utils.clip_grad_norm_(model.parameters(), 5)
          optimizer.step()
      with torch.no_grad():
          h = model.initialise(batchsize)
          testLoss = []
          for x, y in load(testData, batchsize, s):
              x = encode_onehot(x, dim)
              x, y = torch.from_numpy(x), torch.from_numpy(y)
              h = tuple([each.data for each in h])
              inputs, targets = x, y
              inputs, targets = inputs.to(device), targets.to(device)
              predicted, h = model(inputs, h)
              val_loss = criterion(predicted, targets.view(batchsize*s).long())          
              testLoss.append(val_loss.item())
      early_stopping(np.mean(testLoss), model)
      if early_stopping.early_stop:
              print("Early stopping")
              break
      model.load_state_dict(torch.load('checkpoint.pt'))                        
      print("Epoch: {}/{}   ".format(epoch+1, epochs),"Train Loss: {:.4f}".format(np.mean(trainLoss)),"Test Loss: {:.4f}".format(np.mean(testLoss)))

In [10]:
model = RNN_LSTM(len(chars)).to(device)

In [ ]:
train(model, encoded, epochs=100, batchsize=256, s=100)

In [12]:
def test(char, hidden_layer=None, rank=None):  
    inputs = np.array([[intVersion[char]]])
    inputs = encode_onehot(inputs, len(chars))
    inputs = torch.from_numpy(inputs).to(device)        
    hidden_layer_to_tuple = []
    for unit in hidden_layer:
      hidden_layer_to_tuple.append(unit.data)
    hidden_layer_to_tuple = tuple(hidden_layer_to_tuple)
    predicted, hidden_layer = model(inputs, hidden_layer_to_tuple)
    predicted = F.softmax(predicted, dim=1).data.cpu()
    predicted, t = predicted.topk(rank)
    t = np.squeeze(t.numpy())
    predicted = predicted.numpy().squeeze()
    char = np.random.choice(t, p=predicted/predicted.sum())
    return charVersion[char], hidden_layer

In [21]:
!pip install torchsummary
from torchsummary import summary
print(model)

RNN_LSTM(
  (lstm_layer): LSTM(310, 256, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (linear_layer): Linear(in_features=256, out_features=310, bias=True)
)


In [22]:
def writePoem(length, key='The', rank=3):
  with torch.no_grad():    
    poem = [char for char in key]
    hidden_layer = model.initialise(1)
    for c in key:
      char, hidden_layer = test(c, hidden_layer, rank=rank)
    poem.append(char)
    for _ in range(length):
      char, hidden_layer = test(poem[-1], hidden_layer, rank=rank)
      poem.append(char)
  return ''.join(poem)

In [30]:
print(writePoem(length = 1000, key='Sunshine', rank=2))


Sunshine to the stranger of all the winds and the the straicers of a book to the station of the soul of the star of the statues.

They wake to streng that some on to my father, and the body is to see the stars.
The war they were the stone of the straighters and the sea will see the shells of the trees,
the sourness of a sea that was a body of stands and streeches on a chicken.

A stretched of-tree and the second throat in the world, and the water walks
the stars, a change to another and the thing of so ending
to a story of a stranger than the seasons, that wait all that they see
the shadows and then she thinks the things the said in a stracket of the solid
and she had standing their streams and the stood short translations, the strange stars
she watched the street and the sun shoots through a country that wat send
and tractline the black brain and a third stars of a core shell of the sanks
to see his tree and the state of the shining story, the words of the streath

or to the stars to 

In [32]:
print(writePoem(length = 1000, key='Tree', rank=2))

Tree to the stone.

And I ame the tops of the sound
And the shot of the stone starts a soul

The soul was a promonty, a secord that to still
To the flast that the state of the tongue
Of a car the the content of the strength, a breath
And streams are towels, and shakes a star of thinks
To see the water on the shore and till the straight
Shall be an art of stars. The stone wits she says, the world still said,
That the strain will be the sound, the seans in a conden and store,
And that was the body on a thing of the seas of the track.

I wanted the books of a care of the stars, and the same state of the sea,
I saw the body of this to the way they are an and thighs, a strange shade
They start and they say, that to still say it is and the stream of sound
In the stars are a concert to still seem an end of things to the sea of ale the star,
And we think the sound of the stream too much and the shadows and shot to man stink themselves.
They were a shoulder of their song and the stars of the sa

In [37]:
print(writePoem(length = 1000, key='The', rank=2))

The Son
And the steam,
And the stranded through the soul
And the traces of stars.

The shape is the tree,
And all a treacherical streets
The station said. The content of him
A sound of the stream of soldier, the book
And the stars of him and stone
A shallow south of the stranger,
They were searched to the shadow.
The shore with the body of the state
Of a street and stone and stars,
And the company of high streets
And the shoulder to see,
And when the stars and sheets around to set the streets,
The blood on their strains all the sea
That the beneath the sea stones a star,
And shall be the streaming stars,
To star the stream of the stream.
The words are the back of the stone, and the storm
Of the shadows of the way that she was and too some still soul,
And when I stopped a beard and the blank take of the window
The cold star and the street and too shoulders and things.
The warm thiss that was a brick and the stars

The body and so that was a bone of the stair.
They still bring the stars 